## Нелинейный шаг при покупке и продаже v2 #84

In [6]:
%reload_ext autoreload
%autoreload 2

In [14]:
from app.helper import LocalCache
from bot.vis import TaskProgress
from bot import TestAlgorithm
from app.config import RunConfig


from concurrent.futures import ThreadPoolExecutor, as_completed

TICKER = 'RNFT'

test_configs = [
    (RunConfig(
        ticker=TICKER,
        step_max_cnt=max_shares,
        step_base_cnt=base_shares,
        step_lots=2,

        majority_trade=True,
        pretest_period=pretest_period,
        pretest_type=pretest_type,

        threshold_buy_steps=threshold_b_n,
        threshold_sell_steps=threshold_s_n,
        stop_up_p=stop_up_p,
        stop_down_p=stop_down_p,
        step_size_shift=step_size_shift,

        step_size=step_size,
        step_set_orders_cnt=step_cnt,
    ))
    # for max_shares in [3, 4]
    # for base_shares in [0]
    # for threshold_b_n in [0]
    # for threshold_s_n in [0]
    # for step_size in [1.6]
    # for step_cnt in [3]
    # for pretest_period in [6]  # range(2, 7)
    # for pretest_type in [RunConfig.PRETEST_PRE]
    # for stop_up_p in [0]
    # for stop_down_p in [0]
    # for step_size_shift in [0, 0.1, 0.2, 0.3, 0.4, 1]
    for max_shares in [3, 4, 5]
    for base_shares in [0]
    for threshold_b_n in [0]
    for threshold_s_n in [0]
    for step_size in [0.8, 1, 1.2, 1.4, 1.6]
    for step_cnt in [3]
    for pretest_period in [0]
    for pretest_type in [RunConfig.PRETEST_NONE]
    for step_size_shift in [0.1, 0.2, 0.3, 0.4]
    for stop_up_p in [0]
    for stop_down_p in [0]
]


def run_test(config: RunConfig):
    test_alg = TestAlgorithm(do_printing=False, config=config)
    return test_alg.test(
        last_test_date='2024-08-20',
        test_days_num=30,  # 365,
        shares_count=0,
    )

results = []
LocalCache.clear()
progress = TaskProgress(len(test_configs))

with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in test_configs}
    
    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: float(x['profit_p']), reverse=True)

print()
for item in sorted_results:
    print(item)

Запуск в 03:09
Закончено в 03:10, количество 60, длительность 0:00:48                                              

{'profit': 78.9, 'profit_p': 3.93, 'profit_p_avg': 0.13, 'config': RNFT+ 3/0/3 x l2 x 0.8(+x0.1)¤ , 'op': 613}
{'profit': 103.2, 'profit_p': 3.86, 'profit_p_avg': 0.13, 'config': RNFT+ 4/0/3 x l2 x 0.8(+x0.1)¤ , 'op': 657}
{'profit': 76.1, 'profit_p': 3.8, 'profit_p_avg': 0.13, 'config': RNFT+ 3/0/3 x l2 x 0.8(+x0.2)¤ , 'op': 574}
{'profit': 63.7, 'profit_p': 3.18, 'profit_p_avg': 0.11, 'config': RNFT+ 3/0/3 x l2 x 1.2(+x0.1)¤ , 'op': 280}
{'profit': 84.9, 'profit_p': 3.18, 'profit_p_avg': 0.11, 'config': RNFT+ 4/0/3 x l2 x 0.8(+x0.2)¤ , 'op': 604}
{'profit': 60.3, 'profit_p': 3.01, 'profit_p_avg': 0.1, 'config': RNFT+ 3/0/3 x l2 x 0.8(+x0.3)¤ , 'op': 553}
{'profit': 100.4, 'profit_p': 3.0, 'profit_p_avg': 0.1, 'config': RNFT+ 5/0/3 x l2 x 0.8(+x0.1)¤ , 'op': 662}
{'profit': 56.9, 'profit_p': 2.84, 'profit_p_avg': 0.09, 'config': RNFT+ 3/0/3 x l2 x 1.4(+x0.1)¤ , 'op': 19

### Запуск старого алгоритма
last_test_date='2024-08-20',
test_days_num=30,  # 365,

{'profit': 95.8, 'profit_p': 3.58, 'profit_p_avg': 0.12, 'config': RNFT+ 4/pre6:0/3 x l2 x 1.6¤ , 'op': 204}

{'profit': 40.5, 'profit_p': 2.02, 'profit_p_avg': 0.07, 'config': RNFT+ 3/pre6:0/3 x l2 x 1.6¤ , 'op': 140}
